In [ ]:
import tensorflow as tf
import numpy as np
import tkinter as tk
from tkinter import filedialog
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LSTM, Embedding, Reshape, Input
from tensorflow.keras.models import Model
from pyswarm import pso  # Particle Swarm Optimization

# Load and preprocess image
def extract_features(r"C:\Users\dixit\Desktop\OIP (1).jpg")
    model = tf.keras.Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(229, 229, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(512, activation='relu')
    ])
    
    image = load_img(image_path, target_size=(229, 229))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0) / 255.0
    features = model.predict(image)
    return features.flatten()

# Upload image and extract features
image_path = upload_image()
image_features = extract_features("C:\Users\dixit\Desktop\OIP (1).jpg")

# Example dataset
captions = ["A cat sitting on a mat", "A dog playing in the park", "A beautiful sunset over the ocean"]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(captions)
vocab_size = len(tokenizer.word_index) + 1

sequences = tokenizer.texts_to_sequences(captions)
padded_sequences = pad_sequences(sequences, maxlen=10, padding='post')

def create_caption_model():
    inputs = Input(shape=(512,))  # CNN feature vector
    x = Dense(256, activation='relu')(inputs)
    x = Dropout(0.5)(x)
    x = Reshape((1, 256))(x)
    x = LSTM(256, return_sequences=True)(x)
    x = LSTM(256)(x)
    outputs = Dense(vocab_size, activation='softmax')(x)
    model = Model(inputs, outputs)
    return model

model = create_caption_model()

# Define PSO for Hyperparameter Optimization
def objective_function(params):
    learning_rate, batch_size = params
    batch_size = int(batch_size)
    
    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), metrics=['accuracy'])
    
    history = model.fit(padded_sequences, np.zeros((len(captions), vocab_size)), batch_size=batch_size, epochs=1, verbose=0)
    return -history.history['accuracy'][0]  # Minimize negative accuracy

lb = [0.0001, 8]  # Lower bounds for learning rate and batch size
ub = [0.01, 64]  # Upper bounds

best_params, _ = pso(objective_function, lb, ub)
print("Optimized Learning Rate and Batch Size:", best_params)